# Explore the dataset


In this notebook, we will perform an EDA (Exploratory Data Analysis) on the processed Waymo dataset (data in the `processed` folder). In the first part, you will create a function to display 

In [1]:
from utils import get_dataset

In [25]:
import os
import glob

import matplotlib
matplotlib.use('TkAgg')
            
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.patches import Rectangle
from PIL import Image

In [26]:
paths = glob.glob('data/waymo/training_and_validation/*')
i = 0
#filename = os.path.basename(paths)
print(paths[i])
dataset = get_dataset(paths[i])
print('-----------------------------------------------------------')
print(dataset)

data/waymo/training_and_validation/segment-1005081002024129653_5313_150_5333_150_with_camera_labels.tfrecord
INFO:tensorflow:Reading unweighted datasets: ['data/waymo/training_and_validation/segment-1005081002024129653_5313_150_5333_150_with_camera_labels.tfrecord']
INFO:tensorflow:Reading record datasets for input file: ['data/waymo/training_and_validation/segment-1005081002024129653_5313_150_5333_150_with_camera_labels.tfrecord']
INFO:tensorflow:Number of filenames to read: 1
-----------------------------------------------------------
<DatasetV1Adapter shapes: {image: (None, None, 3), source_id: (), key: (), filename: (), groundtruth_image_confidences: (None,), groundtruth_verified_neg_classes: (None,), groundtruth_not_exhaustive_classes: (None,), groundtruth_boxes: (None, 4), groundtruth_area: (None,), groundtruth_is_crowd: (None,), groundtruth_difficult: (None,), groundtruth_group_of: (None,), groundtruth_weights: (None,), groundtruth_classes: (None,), groundtruth_image_classes: (N

## Write a function to display an image and the bounding boxes

Implement the `display_instances` function below. This function takes a batch as an input and display an image with its corresponding bounding boxes. The only requirement is that the classes should be color coded (eg, vehicles in red, pedestrians in blue, cyclist in green).

In [38]:
def display_instances(batch):
    """
    This function takes a batch from the dataset and display the image with 
    the associated bounding boxes.
    この関数は、データセットからバッチを取得し、関連する境界ボックスとともに画像を表示します。
    """
    # ADD CODE HERE

    ##### 色指定
    # color for different classes
    colormap = {1: [1, 0 , 0], 2: [0, 0, 1], 4: [0, 1, 0]}
    
    ##### サブプロット領域設定。1×2、画像サイズ=(10, 10)
    num_col = 5
    num_row = (len(batch) + num_col -1) // num_col
    f, ax = plt.subplots(num_row, num_col, figsize=(20, 10))
    
    ##### batchのインデックスと中身を反復子にしてループ
    for idx, batch_data in enumerate(batch):
        img = batch_data["image"]
        x = idx // num_col
        y = idx % num_col       
        
        ##### 画像をセット
        ax[x, y].imshow(img)
        
        ##### バウンディボックス
        gt_boxes = batch_data["groundtruth_boxes"]
        gt_classes = batch_data["groundtruth_classes"]
        for bb, obj_class in zip(gt_boxes, gt_classes):
            y1, x1, y2, x2 = bb
            x1 *= img.shape[0]
            y1 *= img.shape[1]
            y2 *= img.shape[0]
            x2 *= img.shape[1]
            rec = Rectangle((x1, y1), x2-x1, y2-y1, facecolor='none', edgecolor=colormap[obj_class])
#            rec = Rectangle((x1, y1), x2-x1, y2-y1, facecolor='none', edgecolor='r')
            ax[x, y].add_patch(rec)
    plt.tight_layout()
    plt.show()



## Display 10 images 

Using the dataset created in the second cell and the function you just coded, display 10 random images with the associated bounding boxes. You can use the methods `take` and `shuffle` on the dataset.

In [40]:
## STUDENT SOLUTION HERE

batch = dataset.shuffle(10).take(10)
display_instances(list(batch.as_numpy_iterator()))


def not_use():
    # color mapping of clasess
    colormap = {1:[1, 0, 0], 2:[0, 1, 0], 4:[0, 0, 1]}
    
    f, ax = plt.subplots(4, 5, figsize=(20, 10))
    
    # 
    for i in range(20):
        x = i % 4
        y = i % 5
        
        filename = os.path.basename(paths[i])
        

## Additional EDA

In this last part, you are free to perform any additional analysis of the dataset. What else would like to know about the data?
For example, think about data distribution. So far, you have only looked at a single file...